In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from public_data.models import CouvertureSol, UsageSol, ArtifCommune
from project.models import Project
from prettytable import PrettyTable

In [ ]:
project = Project.objects.get(pk=4)
project

In [ ]:
headers = ['nom', ]
headers += ArtifCommune.list_attr()

table = PrettyTable(field_names=headers, align="l")

for city in project.cities.all():
    row = [city.name, ]
    row += city.list_artif()
    table.add_row(row)

print(table)

In [ ]:
# initialise le DataFrame avec les lignes
indexes = ArtifCommune.list_attr()
df = pd.DataFrame(0, index=indexes, columns=list())
df

In [ ]:
# on rempli le tableau avec les villes du projet
for city in project.cities.all():
    df[city.name] = city.list_artif()
df

In [ ]:
df.T

In [ ]:
# on réduit le DF aux données qui nous intéressent
year_min = 2014
year_max = 2016


index_to_keep = index_to_sum = index_to_exclude = None

# identification des années supérieures qui ne nous intéresse pas (to exclude)
index_to_exclude = []
for i in indexes:
    year = int(i.split("_")[-1])
    if year > year_max:
        index_to_exclude.append(i)
        
index_to_exclude

In [ ]:
# on identifie les index qui font parties des années à garder
index_to_keep = {f"artif_{y}" for y in range(year_min, year_max + 1)}
index_to_keep = set(indexes).intersection(index_to_keep)
index_to_keep

In [ ]:
# on déduit les années à exclure qu'il va falloir sommer 
index_to_sum = set(indexes) - index_to_keep - set(index_to_exclude)
index_to_sum

In [ ]:
# on construit le df des années que l'on va devoir sommer
df_sum = df.loc[list(index_to_sum)]
df_sum

In [ ]:
df_sum.sum()

In [ ]:
avant = total = None
df_final = df.loc[index_to_keep]
df_final

In [ ]:
# on ajoute la ligne "artificialisée avant"
avant = pd.DataFrame(df_sum.sum(), index=df.columns, columns=[f"Avant {year_min}"])
avant = avant.T
df_final = avant.append(df_final)
df_final

In [ ]:
total = pd.DataFrame(df_final.sum(), index=df.columns, columns=["Total"]).T
df_final = df_final.append(total)
df_final.T

In [ ]:
df2 = df_final.T
df2

In [ ]:
# Add progression column (final - init) / init
progression = (df2.loc[:, "Total"] - df2.loc[:, f"Avant {year_min}"]) / df2.loc[:, f"Avant {year_min}"]
progression

In [ ]:
indexes = [f"artif_{y}" for y in range(year_min, year_max + 1)]
indexes = [f"Avant {year_min}"] + indexes + ["Total", "Progression"]
df_final = df_final.reindex(indexes)
df_final

In [ ]:
df3 = df_final.T
df3["Progression"] = progression
df3

In [ ]:
table = []
for city, row in df3.iterrows():
    table.append({
        "name": city,
        "before": row[0],
        "items": list(row[1:-2]),
        "total": row[-2],
        # "surface": city.surface,
        "progression": f"{row[-1]}%",
    })
table

In [ ]:
df_final

In [ ]:
final = df_final.loc["Total"].sum()
final

In [ ]:
label = f"Avant {year_min}"
init = df_final.loc[label].sum()
init

In [ ]:
global_progression = (final - init) / init
global_progression

In [ ]:
df_final.sum()